In [1]:
#Install Packages
!pip install faiss-cpu

In [2]:
!pip install sentence-transformers

  Using cached sentence_transformers-5.1.1-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached torch-2.8.0-cp311-cp311-win_amd64.whl.metadata (30 kB)
  Using cached huggingface_hub-0.35.3-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
Using cached huggingface_hub-0.35.3-py3-none-any.whl (564 kB)
Using cached tokenizers-0.22.1-cp39-abi3-win_amd64.whl (2.7 MB)
Using cached torch-2.8.0-cp311-cp311-win_amd64.whl (241.4 MB)

   ---------------------------------------- 0/5 [torch]
   ---------------------------------------- 0/5 [torch]
   ---------------------------------------- 0/5 [torch]
   ---------------------------------------- 0/5 [torch]
   ---------------------------------------- 0/5 [torch]
   --------------------------------------

In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [5]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [6]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [7]:
#Step 1 : Create source embeddings for the text column

In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shiva\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
vectors.shape

(8, 768)

In [11]:
dim = vectors.shape[1]
dim

768

In [12]:
#Step 2 : Build a FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)

In [14]:
#Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [15]:
index.add(vectors)

In [16]:
#Step 4 : Encode search text using same encorder and normalize the output vector

In [17]:
search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [18]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [20]:
#Step 5: Search for similar vector in the FAISS index created

In [22]:
distances, I = index.search(svec, k=2)
distances

array([[1.3844838, 1.4039094]], dtype=float32)

In [23]:
I

array([[3, 2]])

In [24]:
#3,2 belongs to fashion in csv file

In [25]:
I.tolist()

[[3, 2]]

In [26]:
row_indices = I.tolist()[0]
row_indices

[3, 2]

In [27]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [28]:
search_query

'I want to buy a polo t-shirt'

In [29]:
#You can see that the two results from the dataframe are similar to a search_query